In [3]:
import sys
import os.path
sys.path.append(os.path.join('..', 'util'))


In [4]:
import set_compiler
set_compiler.install()

Compiling with /usr/local/bin/clang-omp


In [5]:
import pyximport
pyximport.install()

(None, <pyximport.pyximport.PyxImporter at 0x1048a2b50>)

In [6]:
import numpy as np
import pylab

In [7]:
import filtering
from timer import Timer
import threading

In [8]:
def numpy_median(image, iterations=10):
    ''' filter using numpy '''
    for i in range(iterations):
        padded = np.pad(image, 1, mode='edge')
        stacked = np.dstack((padded[:-2,  :-2], padded[:-2,  1:-1], padded[:-2,  2:],
                             padded[1:-1, :-2], padded[1:-1, 1:-1], padded[1:-1, 2:],
                             padded[2:,   :-2], padded[2:,   1:-1], padded[2:,   2:]))
        image = np.median(stacked, axis=2)

    return image

In [118]:
def py_median_3x3(image, iterations=10, num_threads=1):
    ''' repeatedly filter with a 3x3 median '''
    tmpA = image.copy()
    tmpB = np.empty_like(tmpA)
    
    for i in range(iterations):
        filtering.median_3x3(tmpA, tmpB, 0, 1)
        tmpA, tmpB = tmpB, tmpA



#     N = num_threads
#     tmpA2 = image.copy()
#     tmpB2 = np.empty_like(tmpA)
#     for i in range(iterations):  
#         for n in range(num_threads):
#             th = threading.Thread(target=filtering.median_3x3, args=(tmpA2
#                                                                      ,tmpB2, n, N))# 0, 1
#             #skipping along columns using index and step
#             #tmpA2, tmpB2 = tmpB2, tmpA2
#             #th.daemon = True
#             th.start()
#         th.join()
#         tmpA2, tmpB2 = tmpB2, tmpA2
        
        
        
    return tmpA

In [122]:
def py_median_3x3_threading(image, iterations=10, num_threads=1):
    ''' repeatedly filter with a 3x3 median '''
    tmpA = image.copy()
    tmpB = np.empty_like(tmpA)
    
    N = num_threads
    tmpA2 = image.copy()
    tmpB2 = np.empty_like(tmpA)
    for i in range(iterations):  
        for n in range(num_threads):
            th = threading.Thread(target=filtering.median_3x3, args=(tmpA2
                                                                     ,tmpB2, n, N))# 0, 1
            #skipping along columns using index and step
            #tmpA2, tmpB2 = tmpB2, tmpA2
            #th.daemon = True
            th.start()
        #th.join()
        tmpA2, tmpB2 = tmpB2, tmpA2
        
        
        
    return tmpA2

In [123]:
if __name__ == '__main__':
    input_image = np.load('image.npz')['image'].astype(np.float32)

    pylab.gray()

    pylab.imshow(input_image)
    pylab.title('original image')

    pylab.figure()
    pylab.imshow(input_image[1200:1800, 3000:3500])
    pylab.title('before - zoom')

    # verify correctness
    from_cython = py_median_3x3(input_image, 1, 2) #original 2 iter
    from_numpy = numpy_median(input_image, 1)#original 2 iter
    assert np.all(from_cython == from_numpy)
    
    with Timer() as t:
        
        new_image = py_median_3x3_threading(input_image, 10, 8)

    pylab.figure()
    pylab.imshow(new_image[1200:1800, 3000:3500])
    pylab.title('after - zoom')

    print("{} seconds for 10 filter passes.".format(t.interval))
    pylab.show()


1.1991379261 seconds for 10 filter passes.


KeyboardInterrupt: 

In [6]:
import threading
import time
import Queue

def worker(queue, table, locks):
    while True:
        idx, value = queue.get()

        with locks[idx]:
            old_value = table[idx]
            time.sleep(0.01 + idx / 10000.0)  # simulate work
            table[idx] = old_value + value

        # update the amount of work left in the queue
        queue.task_done()

if __name__ == '__main__':
    queue = Queue.Queue()
    table = [0] * 5
    locks = [threading.Lock() for _ in table]

    for threadidx in range(4):
        th = threading.Thread(target=worker,
                              args=(queue, table, locks))
        th.daemon = True  # exit even when this thread is alive
        th.start()

    start = time.time()
    for idx in range(100):
        queue.put((idx % len(table), 1))
    queue.join()  # wait for all messages to be processed
    end = time.time()

    print("Total {} in {} seconds".format(sum(table), end - start))


Total 100 in 0.27994799614 seconds
